In [1]:
import pandas as pd
import os
import geopandas as gpd
import glob
import time
import json
import requests

#경고 없애주기
import warnings
warnings.filterwarnings('ignore')

# 윈도우즈 경로
windows_path = r'C:\Users\ghdau\Desktop\real_estate_dashboard'
# 리눅스 경로로 변환
linux_path = windows_path.replace('\\', '/')

# 현재 플랫폼에 따른 파일 경로 생성
if os.name == 'nt':  # 윈도우 플랫폼
    path = linux_path
    print('Window')
else:  # 리눅스나 다른 플랫폼
    path = '/home/ubuntu/real_estate_dashboard'
    print('Linux')

# 작업 디렉토리 변경
os.chdir(path)

print(os.getcwd())

legal_info_b = pd.read_csv("./0_data/legal_info_b.csv",dtype={'법정동코드' : 'str',
                                                              '법정동시군구코드': 'str','법정동읍면동코드':'str'})
legal_info_b

Window
C:\Users\ghdau\Desktop\real_estate_dashboard


,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,법정동시군구코드,법정동읍면동코드,주소
0,1111010100,서울특별시,종로구,청운동,NaN,19880423,,11110,10100,서울특별시 종로구 청운동
1,1111010200,서울특별시,종로구,신교동,NaN,19880423,,11110,10200,서울특별시 종로구 신교동
2,1111010300,서울특별시,종로구,궁정동,NaN,19880423,,11110,10300,서울특별시 종로구 궁정동
3,1111010400,서울특별시,종로구,효자동,NaN,19880423,,11110,10400,서울특별시 종로구 효자동
4,1111010500,서울특별시,종로구,창성동,NaN,19880423,,11110,10500,서울특별시 종로구 창성동
...,...,...,...,...,...,...,...,...,...,...
20273,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,,50130,32022,제주특별자치도 서귀포시 표선면 하천리
20274,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,,50130,32023,제주특별자치도 서귀포시 표선면 성읍리
20275,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,,50130,32024,제주특별자치도 서귀포시 표선면 가시리
20276,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,,50130,32025,제주특별자치도 서귀포시 표선면 세화리


### 법정동 데이터 끝자리에 00 을 더해줘서 지리데이터와 형식을 똑같이 맞춥니다

In [2]:
legal_info_b['법정동코드_2'] = legal_info_b['법정동코드'].str[:8] + '00'
legal_info_b.head(3)

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,법정동시군구코드,법정동읍면동코드,주소,법정동코드_2
0,1111010100,서울특별시,종로구,청운동,NaN,19880423,,11110,10100,서울특별시 종로구 청운동,1111010100
1,1111010200,서울특별시,종로구,신교동,NaN,19880423,,11110,10200,서울특별시 종로구 신교동,1111010200
2,1111010300,서울특별시,종로구,궁정동,NaN,19880423,,11110,10300,서울특별시 종로구 궁정동,1111010300


### 지리 데이터를 불러옵니다

In [3]:
geo_data = gpd.read_file('./0_data/geoservice/emd.shp' , encoding='utf-8')
geo_data

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry
0,11110101,Cheongun-dong,b'c3bbbfeeb5bf',"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,11110102,Singyo-dong,b'bdc5b1b3b5bf',"POLYGON ((953233.465 1953996.984, 953235.183 1..."
2,11110103,Gungjeong-dong,b'b1c3c1a4b5bf',"POLYGON ((953560.228 1954257.466, 953561.190 1..."
3,11110104,Hyoja-dong,b'c8bfc0dab5bf',"POLYGON ((953519.843 1953890.785, 953518.489 1..."
4,11110105,Changseong-dong,b'c3a2bcbab5bf',"POLYGON ((953516.123 1953734.362, 953516.526 1..."
...,...,...,...,...
5060,50130250,Daejeong-eup,b'b4ebc1a4c0be',"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
5061,50130253,Namwon-eup,b'b3b2bff8c0be',"MULTIPOLYGON (((921259.356 1470626.820, 921220..."
5062,50130259,Seongsan-eup,b'bcbabbeac0be',"MULTIPOLYGON (((944514.344 1490162.400, 944510..."
5063,50130310,Andeok-myeon,b'bec8b4f6b8e9',"MULTIPOLYGON (((889476.251 1469109.037, 889465..."


In [4]:
geo_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5065 entries, 0 to 5064
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   EMD_CD      5065 non-null   object  
 1   EMD_ENG_NM  5065 non-null   object  
 2   EMD_KOR_NM  5065 non-null   object  
 3   geometry    5065 non-null   geometry
dtypes: geometry(1), object(3)
memory usage: 158.4+ KB


In [5]:
geo_data.isnull().sum()

EMD_CD        0
EMD_ENG_NM    0
EMD_KOR_NM    0
geometry      0
dtype: int64

In [6]:
geo_data['EMD_CD'] = geo_data['EMD_CD'] + '00'
geo_data['EMD_KOR_NM'] = geo_data['EMD_KOR_NM'] .str.decode('euc-kr')

geo_data.rename(columns={'EMD_CD' : '법정동코드_2'} , inplace=True)
geo_data = geo_data[['법정동코드_2','geometry']]
geo_data

,법정동코드_2,geometry
0,1111010100,"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,1111010200,"POLYGON ((953233.465 1953996.984, 953235.183 1..."
2,1111010300,"POLYGON ((953560.228 1954257.466, 953561.190 1..."
3,1111010400,"POLYGON ((953519.843 1953890.785, 953518.489 1..."
4,1111010500,"POLYGON ((953516.123 1953734.362, 953516.526 1..."
...,...,...
5060,5013025000,"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
5061,5013025300,"MULTIPOLYGON (((921259.356 1470626.820, 921220..."
5062,5013025900,"MULTIPOLYGON (((944514.344 1490162.400, 944510..."
5063,5013031000,"MULTIPOLYGON (((889476.251 1469109.037, 889465..."


In [7]:
geo_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5065 entries, 0 to 5064
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   법정동코드_2   5065 non-null   object  
 1   geometry  5065 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 79.3+ KB


### 법정동 데이터와 지리 데이터를 병합 시켜줍니다

In [8]:
legal_geo_info = pd.merge(legal_info_b, geo_data,
                         on = '법정동코드_2',
                         how = 'left')

print(len(legal_geo_info))
legal_geo_info.head(3).T

20278


,0,1,2
법정동코드,1111010100,1111010200,1111010300
시도명,서울특별시,서울특별시,서울특별시
시군구명,종로구,종로구,종로구
읍면동명,청운동,신교동,궁정동
동리명,NaN,NaN,NaN
생성일자,19880423,19880423,19880423
말소일자,,,
법정동시군구코드,11110,11110,11110
법정동읍면동코드,10100,10200,10300
주소,서울특별시 종로구 청운동,서울특별시 종로구 신교동,서울특별시 종로구 궁정동


In [9]:
# key : 법정동코드
legal_geo_info_2 = legal_geo_info[['법정동코드','시도명','시군구명','동리명','geometry']].drop_duplicates()
print(f'legal_geo_info_2 : {len(legal_geo_info_2)}')

# key : 법정동시군구코드
legal_geo_info_3 = legal_geo_info[['법정동시군구코드','시도명','시군구명']].drop_duplicates()
print(f'legal_geo_info_3 : {len(legal_geo_info_3)}')
# key : 주소
legal_geo_info_4 = legal_geo_info[['주소','법정동코드','법정동읍면동코드','geometry']].drop_duplicates()
print(f'legal_geo_info_4 : {len(legal_geo_info_4)}')

legal_geo_info_2 : 20278
legal_geo_info_3 : 250
legal_geo_info_4 : 20278


In [10]:
# legal_geo_info_2.to_csv("0_data/legal_geo_info_2.csv",index=False)
# legal_geo_info_3.to_csv("0_data/legal_geo_info_3.csv",index=False)
# legal_geo_info_4.to_csv("0_data/legal_geo_info_4.csv",index=False)

### 국토교통부 데이터를 처리합니다
### 아파트 매매 데이터 전처리

In [11]:
apt_trade = pd.read_csv('./0_data/apt_trade/apt_trade_202101.csv' , thousands=r',')
apt_rent = pd.read_csv('./0_data/apt_rent/apt_rent_202101.csv' , thousands=r',')

offi_trade = pd.read_csv('./0_data/offi_trade/offi_trade_202101.csv', thousands=r',')
offi_rent = pd.read_csv('./0_data/offi_rent/offi_rent_202101.csv', thousands=r',')

print(len(apt_trade))
apt_trade.head(3).T

60133


,0,1,2
거래금액,130000,150000,175000
거래유형,,,
건축년도,2000,2008,2004
년,2021,2021,2021
도로명,자하문로33길,경희궁길,사직로8길
도로명건물본번호코드,43,57,24
도로명건물부번호코드,0,0,0
도로명시군구코드,11110,11110,11110
도로명일련번호코드,1,1,5
도로명지상지하코드,0,0,0


In [12]:
apt_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60133 entries, 0 to 60132
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   거래금액        60133 non-null  int64  
 1   거래유형        60133 non-null  object 
 2   건축년도        60133 non-null  int64  
 3   년           60133 non-null  int64  
 4   도로명         60133 non-null  object 
 5   도로명건물본번호코드  60133 non-null  int64  
 6   도로명건물부번호코드  60133 non-null  int64  
 7   도로명시군구코드    60133 non-null  int64  
 8   도로명일련번호코드   60133 non-null  int64  
 9   도로명지상지하코드   60133 non-null  object 
 10  도로명코드       60133 non-null  int64  
 11  등기일자        60133 non-null  object 
 12  법정동         60133 non-null  object 
 13  법정동본번코드     60133 non-null  object 
 14  법정동부번코드     60133 non-null  int64  
 15  법정동시군구코드    60133 non-null  int64  
 16  법정동읍면동코드    60133 non-null  int64  
 17  법정동지번코드     60133 non-null  int64  
 18  아파트         60133 non-null  object 
 19  월           60133 non-nul

In [13]:
apt_trade.isnull().sum()

거래금액          0
거래유형          0
건축년도          0
년             0
도로명           0
도로명건물본번호코드    0
도로명건물부번호코드    0
도로명시군구코드      0
도로명일련번호코드     0
도로명지상지하코드     0
도로명코드         0
등기일자          0
법정동           0
법정동본번코드       0
법정동부번코드       0
법정동시군구코드      0
법정동읍면동코드      0
법정동지번코드       0
아파트           0
월             0
일             0
일련번호          0
전용면적          0
중개사소재지        0
지번            0
지역코드          0
층             0
dtype: int64

### 사용하는 컬럼 만 재정의 해줍니다

In [14]:
apt_trade['법정동코드'] = apt_trade['법정동시군구코드'].astype(str) + apt_trade['법정동읍면동코드'].astype(str)
apt_trade = apt_trade[['거래금액','거래유형','건축년도','전용면적','법정동','법정동코드','아파트','층','년','월','일']]

print(len(apt_trade))
apt_trade.head(3).T

60133


,0,1,2
거래금액,130000,150000,175000
거래유형,,,
건축년도,2000,2008,2004
전용면적,129.76,144.52,174.55
법정동,청운동,사직동,내수동
법정동코드,1111010100,1111011500,1111011800
아파트,청운현대,광화문스페이스본(106동),경희궁의아침2단지
층,2,6,4
년,2021,2021,2021
월,1,1,1


In [15]:
apt_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60133 entries, 0 to 60132
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   거래금액    60133 non-null  int64  
 1   거래유형    60133 non-null  object 
 2   건축년도    60133 non-null  int64  
 3   전용면적    60133 non-null  float64
 4   법정동     60133 non-null  object 
 5   법정동코드   60133 non-null  object 
 6   아파트     60133 non-null  object 
 7   층       60133 non-null  int64  
 8   년       60133 non-null  int64  
 9   월       60133 non-null  int64  
 10  일       60133 non-null  int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 5.0+ MB


In [16]:
apt_trade_final = pd.merge(apt_trade,legal_geo_info_2,
                          on = '법정동코드',
                          how ='left')

apt_trade_final

,거래금액,거래유형,건축년도,전용면적,법정동,법정동코드,아파트,층,년,월,일,시도명,시군구명,동리명,geometry
0,130000,,2000,129.7600,청운동,1111010100,청운현대,2,2021,1,14,서울특별시,종로구,NaN,"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,150000,,2008,144.5200,사직동,1111011500,광화문스페이스본(106동),6,2021,1,7,서울특별시,종로구,NaN,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
2,175000,,2004,174.5500,내수동,1111011800,경희궁의아침2단지,4,2021,1,15,서울특별시,종로구,NaN,"POLYGON ((953504.696 1952866.960, 953505.697 1..."
3,175000,,2004,174.5500,내수동,1111011800,경희궁의아침2단지,4,2021,1,15,서울특별시,종로구,NaN,"POLYGON ((953504.696 1952866.960, 953505.697 1..."
4,180000,,2004,123.1300,내수동,1111011800,경희궁의아침3단지,13,2021,1,17,서울특별시,종로구,NaN,"POLYGON ((953504.696 1952866.960, 953505.697 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60128,22800,,2015,69.0088,성산읍 고성리,5013025924,드림펠리스,4,2021,1,21,제주특별자치도,서귀포시,고성리,"MULTIPOLYGON (((944514.344 1490162.400, 944510..."
60129,23000,,2017,84.8200,안덕면 화순리,5013031021,엘림피렌체3차,4,2021,1,26,제주특별자치도,서귀포시,화순리,"MULTIPOLYGON (((889476.251 1469109.037, 889465..."
60130,23000,,2017,84.8200,안덕면 화순리,5013031021,엘림피렌체3차,4,2021,1,26,제주특별자치도,서귀포시,화순리,"MULTIPOLYGON (((889476.251 1469109.037, 889465..."
60131,23000,,2017,84.8200,안덕면 화순리,5013031021,엘림피렌체3차,4,2021,1,26,제주특별자치도,서귀포시,화순리,"MULTIPOLYGON (((889476.251 1469109.037, 889465..."


In [17]:
apt_trade_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60133 entries, 0 to 60132
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   거래금액      60133 non-null  int64   
 1   거래유형      60133 non-null  object  
 2   건축년도      60133 non-null  int64   
 3   전용면적      60133 non-null  float64 
 4   법정동       60133 non-null  object  
 5   법정동코드     60133 non-null  object  
 6   아파트       60133 non-null  object  
 7   층         60133 non-null  int64   
 8   년         60133 non-null  int64   
 9   월         60133 non-null  int64   
 10  일         60133 non-null  int64   
 11  시도명       60133 non-null  object  
 12  시군구명      59678 non-null  object  
 13  동리명       9501 non-null   object  
 14  geometry  60079 non-null  geometry
dtypes: float64(1), geometry(1), int64(6), object(7)
memory usage: 6.9+ MB


In [18]:
apt_trade_final.isnull().sum()

거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          455
동리명         50632
geometry       54
dtype: int64

In [19]:
apt_trade_final[apt_trade_final['시군구명'].isnull()]['시도명'].unique()

array(['세종특별자치시'], dtype=object)

### 아파트 전월세 데이터 전처리
### 10자리 법정동코드가 없고 대신 지역코드 5자리만 있습니다

In [20]:
print(len(apt_rent))
apt_rent.head(3).T

74583


,0,1,2
갱신요구권사용,,,
건축년도,2008,2008,2008
계약구분,,,
계약기간,,,
년,2021,2021,2021
법정동,사직동,사직동,사직동
보증금액,20000,3000,50000
아파트,광화문스페이스본(106동),광화문스페이스본(101동~105동),광화문스페이스본(101동~105동)
월,1,1,1
월세금액,280,300,150


### 1. 지역코드를 이용하여 시도명 시군구명을 얻어옵니다
### 2. 시도명과 시군구명 법정동을 더하여 주소 칼럼을 만듭니다
### 3. 미리만들어둔 legal_geo_info_4 주소 칼럼을 기준으로 최종 병합 합니다

### Step 1

In [21]:
# 필요한 컬럼만 가져와 재정의 해줍니다
apt_rent = apt_rent[['건축년도','법정동','지번','아파트','보증금액','월세금액','전용면적','층','지역코드','년','월','일']]

apt_rent.rename(columns = {'지역코드':'법정동시군구코드'}, inplace=True)
apt_rent.rename(columns = {'법정동':'동리명'}, inplace=True)

In [22]:
apt_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74583 entries, 0 to 74582
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   건축년도      74583 non-null  int64  
 1   동리명       74583 non-null  object 
 2   지번        74583 non-null  object 
 3   아파트       74583 non-null  object 
 4   보증금액      74583 non-null  int64  
 5   월세금액      74583 non-null  int64  
 6   전용면적      74583 non-null  float64
 7   층         74583 non-null  int64  
 8   법정동시군구코드  74583 non-null  int64  
 9   년         74583 non-null  int64  
 10  월         74583 non-null  int64  
 11  일         74583 non-null  int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 6.8+ MB


In [23]:
apt_rent = apt_rent.astype({'법정동시군구코드':'str'})

apt_rent_2 = pd.merge(apt_rent,legal_geo_info_3,
                    on = '법정동시군구코드',
                     how = 'left')
apt_rent_2

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명
0,2008,사직동,9-1,광화문스페이스본(106동),20000,280,150.4000,5,11110,2021,1,5,서울특별시,종로구
1,2008,사직동,9,광화문스페이스본(101동~105동),3000,300,94.5100,9,11110,2021,1,7,서울특별시,종로구
2,2008,사직동,9,광화문스페이스본(101동~105동),50000,150,108.5500,1,11110,2021,1,9,서울특별시,종로구
3,2008,사직동,9,광화문스페이스본(101동~105동),80000,0,70.8000,12,11110,2021,1,9,서울특별시,종로구
4,2008,사직동,9,광화문스페이스본(101동~105동),120000,0,158.9900,13,11110,2021,1,12,서울특별시,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74578,2010,대정읍 하모리,1450,서귀포대정휴먼시아,1439,12,46.8000,5,50130,2021,1,18,제주특별자치도,서귀포시
74579,2019,대정읍 하모리,1089-22,비슬휴림,1000,67,65.8886,5,50130,2021,1,22,제주특별자치도,서귀포시
74580,1996,남원읍 남원리,1323-2,에덴,7000,0,40.0000,3,50130,2021,1,20,제주특별자치도,서귀포시
74581,2020,성산읍 고성리,1141-1,서귀포성산엘에이치아파트,699,12,26.4400,7,50130,2021,1,2,제주특별자치도,서귀포시


### Step 2

In [24]:
#  먼저 각 컬럼에 공백을 제거해줍니다
apt_rent_2['시도명'] = apt_rent_2['시도명'].str.strip()
apt_rent_2['시군구명'] = apt_rent_2['시군구명'].str.strip()
apt_rent_2['동리명'] = apt_rent_2['동리명'].str.strip()

apt_rent_2['주소'] = apt_rent_2['시도명'] + " " + apt_rent_2['시군구명'] + " " + apt_rent_2['동리명']

### Step 3

In [25]:
apt_rent_3 = pd.merge(apt_rent_2, legal_geo_info_4,
                     on = '주소',
                     how = 'left')
apt_rent_3

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry
0,2008,사직동,9-1,광화문스페이스본(106동),20000,280,150.4000,5,11110,2021,1,5,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
1,2008,사직동,9,광화문스페이스본(101동~105동),3000,300,94.5100,9,11110,2021,1,7,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
2,2008,사직동,9,광화문스페이스본(101동~105동),50000,150,108.5500,1,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
3,2008,사직동,9,광화문스페이스본(101동~105동),80000,0,70.8000,12,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
4,2008,사직동,9,광화문스페이스본(101동~105동),120000,0,158.9900,13,11110,2021,1,12,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74578,2010,대정읍 하모리,1450,서귀포대정휴먼시아,1439,12,46.8000,5,50130,2021,1,18,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리,5013025022,25022,"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
74579,2019,대정읍 하모리,1089-22,비슬휴림,1000,67,65.8886,5,50130,2021,1,22,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리,5013025022,25022,"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
74580,1996,남원읍 남원리,1323-2,에덴,7000,0,40.0000,3,50130,2021,1,20,제주특별자치도,서귀포시,제주특별자치도 서귀포시 남원읍 남원리,5013025321,25321,"MULTIPOLYGON (((921259.356 1470626.820, 921220..."
74581,2020,성산읍 고성리,1141-1,서귀포성산엘에이치아파트,699,12,26.4400,7,50130,2021,1,2,제주특별자치도,서귀포시,제주특별자치도 서귀포시 성산읍 고성리,5013025924,25924,"MULTIPOLYGON (((944514.344 1490162.400, 944510..."


In [26]:
apt_rent_3.isnull().sum()

건축년도           0
동리명            0
지번             0
아파트            0
보증금액           0
월세금액           0
전용면적           0
층              0
법정동시군구코드       0
년              0
월              0
일              0
시도명            0
시군구명        1121
주소          1121
법정동코드       1145
법정동읍면동코드    1145
geometry    1154
dtype: int64

### 하지만 병합을 하고 보니 법정동코드에 공백 컬럼이 보입니다

In [27]:
apt_rent_3[apt_rent_3['법정동코드'].isnull()]

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry
34882,2017,소담동,598,새샘마을5단지(한양수자인엘시티),20000,0,59.4544,8,36110,2021,1,20,세종특별자치시,NaN,NaN,NaN,NaN,None
34883,2017,소담동,598,새샘마을5단지(한양수자인엘시티),18000,0,84.6347,17,36110,2021,1,22,세종특별자치시,NaN,NaN,NaN,NaN,None
34884,2017,소담동,598,새샘마을5단지(한양수자인엘시티),2000,70,59.4544,11,36110,2021,1,27,세종특별자치시,NaN,NaN,NaN,NaN,None
34885,2016,소담동,521,새샘마을9단지(중흥S클래스리버뷰),37000,0,84.9779,15,36110,2021,1,28,세종특별자치시,NaN,NaN,NaN,NaN,None
34886,2017,보람동,783,호려울마을5단지(제일풍경채),40000,0,99.9860,10,36110,2021,1,2,세종특별자치시,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58834,2020,북문로3가동,123,청주행정타운코아루휴티스,29000,0,84.6871,34,43111,2021,1,30,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58835,2020,북문로3가동,123,청주행정타운코아루휴티스,25000,0,84.6871,9,43111,2021,1,30,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58836,2020,북문로3가동,123,청주행정타운코아루휴티스,25000,0,84.6871,47,43111,2021,1,31,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58837,2016,남문로1가동,53,트레비앙,1000,30,22.5400,9,43111,2021,1,20,충청북도,청주시 상당구,충청북도 청주시 상당구 남문로1가동,NaN,NaN,None


In [28]:
apt_rent_3[apt_rent_3['법정동코드'].isnull()][['주소','시도명']]

,주소,시도명
34882,NaN,세종특별자치시
34883,NaN,세종특별자치시
34884,NaN,세종특별자치시
34885,NaN,세종특별자치시
34886,NaN,세종특별자치시
...,...,...
58834,충청북도 청주시 상당구 북문로3가동,충청북도
58835,충청북도 청주시 상당구 북문로3가동,충청북도
58836,충청북도 청주시 상당구 북문로3가동,충청북도
58837,충청북도 청주시 상당구 남문로1가동,충청북도


### 세종특별자치시와 충청북도 칼럼이 문제입니다

In [29]:
apt_rent_3[apt_rent_3['법정동코드'].isnull()][['주소','시도명']].drop_duplicates()

,주소,시도명
34882,NaN,세종특별자치시
58815,충청북도 청주시 상당구 북문로2가동,충청북도
58818,충청북도 청주시 상당구 북문로3가동,충청북도
58837,충청북도 청주시 상당구 남문로1가동,충청북도


### 세종특별자치시는 시군구명 값이 없습니다

In [30]:
apt_rent_3[apt_rent_3['법정동시군구코드'] == '36110']

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry
34882,2017,소담동,598,새샘마을5단지(한양수자인엘시티),20000,0,59.4544,8,36110,2021,1,20,세종특별자치시,NaN,NaN,NaN,NaN,None
34883,2017,소담동,598,새샘마을5단지(한양수자인엘시티),18000,0,84.6347,17,36110,2021,1,22,세종특별자치시,NaN,NaN,NaN,NaN,None
34884,2017,소담동,598,새샘마을5단지(한양수자인엘시티),2000,70,59.4544,11,36110,2021,1,27,세종특별자치시,NaN,NaN,NaN,NaN,None
34885,2016,소담동,521,새샘마을9단지(중흥S클래스리버뷰),37000,0,84.9779,15,36110,2021,1,28,세종특별자치시,NaN,NaN,NaN,NaN,None
34886,2017,보람동,783,호려울마을5단지(제일풍경채),40000,0,99.9860,10,36110,2021,1,2,세종특별자치시,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35998,2000,금남면 용포리,320,두진리버빌,300,32,43.0080,14,36110,2021,1,27,세종특별자치시,NaN,NaN,NaN,NaN,None
35999,2002,장군면 금암리,259-1,영조정보마을임대아파트,3200,0,28.2616,1,36110,2021,1,6,세종특별자치시,NaN,NaN,NaN,NaN,None
36000,1992,전의면 동교리,197-16,재동,4500,0,59.8700,5,36110,2021,1,20,세종특별자치시,NaN,NaN,NaN,NaN,None
36001,1991,전의면 동교리,121-6,"재동(1,2동)",5000,0,59.8800,1,36110,2021,1,22,세종특별자치시,NaN,NaN,NaN,NaN,None


In [31]:
apt_rent_2.where(pd.notnull(apt_rent_2),'')

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소
0,2008,사직동,9-1,광화문스페이스본(106동),20000,280,150.4000,5,11110,2021,1,5,서울특별시,종로구,서울특별시 종로구 사직동
1,2008,사직동,9,광화문스페이스본(101동~105동),3000,300,94.5100,9,11110,2021,1,7,서울특별시,종로구,서울특별시 종로구 사직동
2,2008,사직동,9,광화문스페이스본(101동~105동),50000,150,108.5500,1,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동
3,2008,사직동,9,광화문스페이스본(101동~105동),80000,0,70.8000,12,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동
4,2008,사직동,9,광화문스페이스본(101동~105동),120000,0,158.9900,13,11110,2021,1,12,서울특별시,종로구,서울특별시 종로구 사직동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74578,2010,대정읍 하모리,1450,서귀포대정휴먼시아,1439,12,46.8000,5,50130,2021,1,18,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리
74579,2019,대정읍 하모리,1089-22,비슬휴림,1000,67,65.8886,5,50130,2021,1,22,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리
74580,1996,남원읍 남원리,1323-2,에덴,7000,0,40.0000,3,50130,2021,1,20,제주특별자치도,서귀포시,제주특별자치도 서귀포시 남원읍 남원리
74581,2020,성산읍 고성리,1141-1,서귀포성산엘에이치아파트,699,12,26.4400,7,50130,2021,1,2,제주특별자치도,서귀포시,제주특별자치도 서귀포시 성산읍 고성리


In [32]:
apt_rent_2['시도명'] = apt_rent_2['시도명'].str.strip()
apt_rent_2['시군구명'] = apt_rent_2['시군구명'].str.strip()
apt_rent_2['동리명'] = apt_rent_2['동리명'].str.strip()

apt_rent_2 = apt_rent_2.where(pd.notnull(apt_rent_2),'')

apt_rent_2['주소'] = apt_rent_2['시도명'] + " " + apt_rent_2['시군구명'] + " " + apt_rent_2['동리명']

apt_rent_2['주소'] = apt_rent_2['주소'].str.replace('  ',' ')
apt_rent_2['주소'] = apt_rent_2['주소'].str.strip()

In [33]:
apt_rent_3 = pd.merge(apt_rent_2, legal_geo_info_4,
                     on = '주소',
                     how = 'left')
apt_rent_3

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry
0,2008,사직동,9-1,광화문스페이스본(106동),20000,280,150.4000,5,11110,2021,1,5,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
1,2008,사직동,9,광화문스페이스본(101동~105동),3000,300,94.5100,9,11110,2021,1,7,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
2,2008,사직동,9,광화문스페이스본(101동~105동),50000,150,108.5500,1,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
3,2008,사직동,9,광화문스페이스본(101동~105동),80000,0,70.8000,12,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
4,2008,사직동,9,광화문스페이스본(101동~105동),120000,0,158.9900,13,11110,2021,1,12,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74578,2010,대정읍 하모리,1450,서귀포대정휴먼시아,1439,12,46.8000,5,50130,2021,1,18,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리,5013025022,25022,"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
74579,2019,대정읍 하모리,1089-22,비슬휴림,1000,67,65.8886,5,50130,2021,1,22,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리,5013025022,25022,"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
74580,1996,남원읍 남원리,1323-2,에덴,7000,0,40.0000,3,50130,2021,1,20,제주특별자치도,서귀포시,제주특별자치도 서귀포시 남원읍 남원리,5013025321,25321,"MULTIPOLYGON (((921259.356 1470626.820, 921220..."
74581,2020,성산읍 고성리,1141-1,서귀포성산엘에이치아파트,699,12,26.4400,7,50130,2021,1,2,제주특별자치도,서귀포시,제주특별자치도 서귀포시 성산읍 고성리,5013025924,25924,"MULTIPOLYGON (((944514.344 1490162.400, 944510..."


In [34]:
apt_rent_3[apt_rent_3['법정동코드'].isnull()].drop_duplicates()

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry
58815,2016,북문로2가동,78-10,아트에비뉴198,200,30,18.9750,8,43111,2021,1,7,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로2가동,NaN,NaN,None
58816,2016,북문로2가동,78-10,아트에비뉴198,300,30,18.9750,7,43111,2021,1,16,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로2가동,NaN,NaN,None
58817,2016,북문로2가동,78-10,아트에비뉴198,300,30,21.8670,12,43111,2021,1,17,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로2가동,NaN,NaN,None
58818,2020,북문로3가동,123,청주행정타운코아루휴티스,28000,0,84.9935,27,43111,2021,1,4,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58819,2020,북문로3가동,123,청주행정타운코아루휴티스,3000,70,84.9935,20,43111,2021,1,12,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58820,2020,북문로3가동,123,청주행정타운코아루휴티스,30000,0,84.9935,28,43111,2021,1,14,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58821,2020,북문로3가동,123,청주행정타운코아루휴티스,29000,0,84.9935,31,43111,2021,1,14,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58822,2020,북문로3가동,123,청주행정타운코아루휴티스,26000,0,84.9935,8,43111,2021,1,17,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58823,2020,북문로3가동,123,청주행정타운코아루휴티스,27000,0,84.9935,20,43111,2021,1,18,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None
58824,2020,북문로3가동,123,청주행정타운코아루휴티스,25000,0,84.6871,19,43111,2021,1,18,충청북도,청주시 상당구,충청북도 청주시 상당구 북문로3가동,NaN,NaN,None


### 충청북도 청주시 상당구 뒤에 동이 없기때문에 수정해줍니다 

In [35]:
apt_rent_2 = apt_rent_2.replace('충청북도 청주시 상당구 북문로2가동','충청북도 청주시 상당구 북문로2가')
apt_rent_2 = apt_rent_2.replace('충청북도 청주시 상당구 북문로3가동','충청북도 청주시 상당구 북문로3가')
apt_rent_2 = apt_rent_2.replace('충청북도 청주시 상당구 남문로1가동','충청북도 청주시 상당구 남문로1가')

### 수정된 데이터 프레임을 다시 병합해보면 문제가 없음을 알 수 있습니다

In [36]:
apt_rent_3 = pd.merge(apt_rent_2, legal_geo_info_4,
                     on = '주소',
                     how = 'left')
apt_rent_3

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry
0,2008,사직동,9-1,광화문스페이스본(106동),20000,280,150.4000,5,11110,2021,1,5,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
1,2008,사직동,9,광화문스페이스본(101동~105동),3000,300,94.5100,9,11110,2021,1,7,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
2,2008,사직동,9,광화문스페이스본(101동~105동),50000,150,108.5500,1,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
3,2008,사직동,9,광화문스페이스본(101동~105동),80000,0,70.8000,12,11110,2021,1,9,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
4,2008,사직동,9,광화문스페이스본(101동~105동),120000,0,158.9900,13,11110,2021,1,12,서울특별시,종로구,서울특별시 종로구 사직동,1111011500,11500,"POLYGON ((952745.849 1953334.909, 952758.791 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74578,2010,대정읍 하모리,1450,서귀포대정휴먼시아,1439,12,46.8000,5,50130,2021,1,18,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리,5013025022,25022,"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
74579,2019,대정읍 하모리,1089-22,비슬휴림,1000,67,65.8886,5,50130,2021,1,22,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 하모리,5013025022,25022,"MULTIPOLYGON (((884996.737 1458754.044, 884991..."
74580,1996,남원읍 남원리,1323-2,에덴,7000,0,40.0000,3,50130,2021,1,20,제주특별자치도,서귀포시,제주특별자치도 서귀포시 남원읍 남원리,5013025321,25321,"MULTIPOLYGON (((921259.356 1470626.820, 921220..."
74581,2020,성산읍 고성리,1141-1,서귀포성산엘에이치아파트,699,12,26.4400,7,50130,2021,1,2,제주특별자치도,서귀포시,제주특별자치도 서귀포시 성산읍 고성리,5013025924,25924,"MULTIPOLYGON (((944514.344 1490162.400, 944510..."


In [37]:
apt_rent_3[apt_rent_3['법정동시군구코드'] == '36110']

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry
34882,2017,소담동,598,새샘마을5단지(한양수자인엘시티),20000,0,59.4544,8,36110,2021,1,20,세종특별자치시,,세종특별자치시 소담동,3611010200,10200,"POLYGON ((982515.836 1832329.682, 982521.520 1..."
34883,2017,소담동,598,새샘마을5단지(한양수자인엘시티),18000,0,84.6347,17,36110,2021,1,22,세종특별자치시,,세종특별자치시 소담동,3611010200,10200,"POLYGON ((982515.836 1832329.682, 982521.520 1..."
34884,2017,소담동,598,새샘마을5단지(한양수자인엘시티),2000,70,59.4544,11,36110,2021,1,27,세종특별자치시,,세종특별자치시 소담동,3611010200,10200,"POLYGON ((982515.836 1832329.682, 982521.520 1..."
34885,2016,소담동,521,새샘마을9단지(중흥S클래스리버뷰),37000,0,84.9779,15,36110,2021,1,28,세종특별자치시,,세종특별자치시 소담동,3611010200,10200,"POLYGON ((982515.836 1832329.682, 982521.520 1..."
34886,2017,보람동,783,호려울마을5단지(제일풍경채),40000,0,99.9860,10,36110,2021,1,2,세종특별자치시,,세종특별자치시 보람동,3611010300,10300,"POLYGON ((981533.190 1831957.290, 981530.993 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35998,2000,금남면 용포리,320,두진리버빌,300,32,43.0080,14,36110,2021,1,27,세종특별자치시,,세종특별자치시 금남면 용포리,3611034021,34021,"POLYGON ((987548.917 1832590.855, 987513.919 1..."
35999,2002,장군면 금암리,259-1,영조정보마을임대아파트,3200,0,28.2616,1,36110,2021,1,6,세종특별자치시,,세종특별자치시 장군면 금암리,3611035025,35025,"POLYGON ((976668.256 1832051.548, 976681.266 1..."
36000,1992,전의면 동교리,197-16,재동,4500,0,59.8700,5,36110,2021,1,20,세종특별자치시,,세종특별자치시 전의면 동교리,3611037022,37022,"POLYGON ((972819.479 1845225.865, 972814.966 1..."
36001,1991,전의면 동교리,121-6,"재동(1,2동)",5000,0,59.8800,1,36110,2021,1,22,세종특별자치시,,세종특별자치시 전의면 동교리,3611037022,37022,"POLYGON ((972819.479 1845225.865, 972814.966 1..."


In [38]:
apt_rent_3[apt_rent_3['법정동코드'].isnull()].drop_duplicates()

,건축년도,동리명,지번,아파트,보증금액,월세금액,전용면적,층,법정동시군구코드,년,월,일,시도명,시군구명,주소,법정동코드,법정동읍면동코드,geometry


### 다른 데이터 컬럼도 살펴보겠습니다

In [39]:
print(offi_trade.columns)
print(offi_rent.columns)

Index(['거래금액', '거래유형', '건축년도', '년', '단지', '법정동', '시군구', '월', '일', '전용면적',
       '중개사소재지', '지번', '지역코드', '층'],
      dtype='object')
Index(['갱신요구권사용', '건축년도', '계약구분', '계약기간', '년', '단지', '법정동', '보증금', '시군구', '월',
       '월세', '일', '전용면적', '종전계약보증금', '종전계약월세', '지번', '지역코드', '층'],
      dtype='object')


### 나머지 데이터는 아파트 전월세 데이터 처럼 지역코드가 있는 경우 이므로
### 함수를 구현하여 빠르게 전처리를 하겠습니다

In [40]:
def molit_data_cleaning(df,df_name):
    
    # 아파트 실거래 매매 데이터 인 경우
    if('법정동시군구코드' in df.columns):
        print(df_name)
        df = df.astype({'법정동시군구코드' : 'str',
                        '법정동읍면동코드' : 'str'})
        df['법정동코드'] = df['법정동시군구코드'] + df['법정동읍면동코드']
        df = df[['거래금액','거래유형','건축년도','전용면적','법정동','법정동코드','아파트',
                '층','년','월','일']]
        
        df_final = pd.merge(df,legal_geo_info_2,
                           on = '법정동코드',
                           how = 'left')
        
        print('-' * 100)
        print(df_final.isnull().sum())
        print()
        
    # 아파트 실거래 매매 제외 전부    
    else:
        print(df_name)
        # 칼럼 이름 변경
        df.rename(columns={'지역코드':'법정동시군구코드'},inplace=True)
        df.rename(columns={'법정동':'동리명'},inplace=True)
        
        # 칼럼 자료형 변경
        df = df.astype({'법정동시군구코드' : 'str'})
        
        # 키 값 = 법정동시군구코드
        df = pd.merge(df,legal_geo_info_3,
                     on = '법정동시군구코드',
                     how = 'left')
        
        # 컬럼 양쪽 공백 제거
        df['시도명'] = df['시도명'].str.strip()
        df['시군구명'] = df['시군구명'].str.strip()
        df['동리명'] = df['동리명'].str.strip()
        
        # Nan 값을 공백으로 변경
        df = df.where(pd.notnull(df),'')
        
        # 주소 컬럼 추가
        df['주소'] = df['시도명'] + " " + df['시군구명'] + " " + df['동리명']

        # 주소 칼럼 공백 2개 부분을 1개로 변경
        # 주소 양쪽 공백 제거
        df['주소'] = df['주소'].str.replace('  ',' ')
        df['주소'] = df['주소'].str.strip()
        
        # 예외 문자열 변경
        df = df.replace('충청북도 청주시 상당구 북문로2가동','충청북도 청주시 상당구 북문로2가')
        df = df.replace('충청북도 청주시 상당구 북문로3가동','충청북도 청주시 상당구 북문로3가')
        df = df.replace('충청북도 청주시 상당구 남문로1가동','충청북도 청주시 상당구 남문로1가')
        
        # 최종 병합 키 값은 주소
        df_final = pd.merge(df, legal_geo_info_4,
                     on = '주소',
                     how = 'left')
        
        # 결측값을 가진 행 제거
        df_final = df_final.dropna()
        
        print('-' * 100)
        print(df_final.isnull().sum())
        print()
        
    return df_final

### 각 데이터에 함수가 에러가 없는지 테스트 해봅니다

In [41]:
apt_trade = pd.read_csv('./0_data/apt_trade/apt_trade_202103.csv' , thousands=r',')
apt_rent = pd.read_csv('./0_data/apt_rent/apt_rent_202103.csv' , thousands=r',')

offi_trade = pd.read_csv('./0_data/offi_trade/offi_trade_202103.csv', thousands=r',')
offi_rent = pd.read_csv('./0_data/offi_rent/offi_rent_202103.csv', thousands=r',')

datas = [apt_trade,apt_rent,offi_trade,offi_rent]
names = ['아파트 매매','아파트 전월세','오피스텔 매매','오피스텔 전월세']

In [42]:
for data , name in zip(datas,names):
    df_clean = molit_data_cleaning(data, name)
    if(name !=  '아파트 매매'):
        print('법정동코드 : ' , df_clean['법정동코드'].isnull().sum())

아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          407
동리명         49719
geometry       23
dtype: int64

아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

법정동코드 :  0
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년

### 데이터 저장

In [43]:
# 테스트
apt_trade_test = molit_data_cleaning(apt_trade,'아파트 매매')
print(apt_trade_test.shape)

아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          407
동리명         49719
geometry       23
dtype: int64

(59731, 15)


### 지리 정보 때문에 데이터에 크기가 많이 나옵니다

In [44]:
# file_name = path + '/0_data/cleaning/apt_trade_test.csv'
# # file_name = path + '/0_data/cleaning/apt_trade_test.csv''

# apt_trade_test.to_csv(file_name,index = False)

### 지리 정보를 제거하고 저장해 보겠습니다

In [45]:
apt_trade_test_rm_geometry = molit_data_cleaning(apt_trade,'아파트 매매')
apt_trade_test_rm_geometry = apt_trade_test_rm_geometry.drop(columns =['geometry'])

# file_name = './0_data/cleaning/apt_trade_test_rm_geometry.csv'
# file_name = ./0_data/cleaning/apt_trade_test_rm_geometry.csv''

# apt_trade_test_rm_geometry.to_csv(file_name,index = False)

아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          407
동리명         49719
geometry       23
dtype: int64



### geometry 칼럼 문제 해결

### 1. 수집한 모든 데이터를 함수를 통해 전처리 해줍니다
### 2. 전처리후 geomtery 칼럼을 제거한후 이어 붙입니다
### 3. 이어 붙인 데이터는 0_data/streamlit_data/{---}/에 하나씩 저장 합니다

In [46]:
# 실행 전 시간 기록
start_time = time.time()

# 국토교통부 데이터 세트
list_nm =['apt_trade','apt_rent','offi_trade','offi_rent']
names = ['아파트 매매','아파트 전월세','오피스텔 매매','오피스텔 전월세']

# 4개 데이터 세트 반복문
for i in range(len(list_nm)):
    directory = f'0_data/streamlit_data/{list_nm[i]}'
    # 폴더가 없으면 폴더 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"폴더 '{directory}'가 생성되었습니다.")
    
    # 각 타입별 모든 데이터 조회
    dir_list = f'./0_data/{list_nm[i]}/*.csv'
    file_list = sorted(glob.glob(dir_list))

    if os.name == 'nt':  # 윈도우 플랫폼
        file_list = [file.replace('\\', '/') for file in file_list]
    
    for k in range(len(file_list)):
        
        print(k + 1)
        
        total = pd.DataFrame()
        
        # 파일 경로를 디렉토리와 파일 이름으로 분리
        directory, file_name = os.path.split(file_list[k])

        # 파일 이름에서 연월 부분 추출
        year_month = file_name.split('_')[-1].split('.')[0]   
        
        try:
            df = pd.read_csv(file_list[k], thousands=r',')
        
            clean_df = molit_data_cleaning(df,names[i])
            clean_df = clean_df.drop(columns=['geometry'])
        
            total = pd.concat([total,clean_df])
    
            file_name = f'0_data/streamlit_data/{list_nm[i]}/{list_nm[i]}_{year_month}.csv'
            total.to_csv(file_name, index = False)
        except Exception as e:
            print(f"예외가 발생했습니다: {e}")
            break
            
# 실행 후 시간 기록
end_time = time.time()  

# 실행 시간 계산
execution_time = end_time - start_time
print(f"실행 시간: {execution_time} 초")   

1
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          455
동리명         50632
geometry       54
dtype: int64

2
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          375
동리명         42413
geometry       24
dtype: int64

3
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드       

거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          183
동리명         12992
geometry        7
dtype: int64

23
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          232
동리명         12512
geometry        5
dtype: int64

24
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명         

7
아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

8
아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

9
아파트 전월세
------------------------------------------------------------------------------------------------

26
아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

27
아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

28
아파트 전월세
---------------------------------------------------------------------------------------------

오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

11
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

12
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명      

34
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

35
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

36
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명   

----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

18
오피스텔 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

19
오피스텔 전월세
--------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

36
오피스텔 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

실행 시간: 77.5541160106659 초


### 0_data/streamlit_data/{---}/에 종류 별로 통 파일을 저장 합니다

In [47]:
# 실행 전 시간 기록
start_time = time.time()

# 국토교통부 데이터 세트
list_nm =['apt_trade','apt_rent','offi_trade','offi_rent']
names = ['아파트 매매','아파트 전월세','오피스텔 매매','오피스텔 전월세']

# 4개 데이터 세트 반복문
for i in range(len(list_nm)):
    directory = f'0_data/streamlit_data/{list_nm[i]}'
    # 폴더가 없으면 폴더 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"폴더 '{directory}'가 생성되었습니다.")
    
    total = pd.DataFrame()
    
    # 각 타입별 모든 데이터 조회
    dir_list = f'./0_data/{list_nm[i]}/*.csv'
    file_list = sorted(glob.glob(dir_list))

    if os.name == 'nt':  # 윈도우 플랫폼
        file_list = [file.replace('\\', '/') for file in file_list]
    
    for k in range(len(file_list)):
        print(k + 1)     
        
        try:
            df = pd.read_csv(file_list[k], thousands=r',')
        
            clean_df = molit_data_cleaning(df,names[i])
            clean_df = clean_df.drop(columns=['geometry'])
        
            total = pd.concat([total,clean_df])
            print(len(total))
        except Exception as e:
            print(f"예외가 발생했습니다: {e}")
            break
            
    file_name = f'0_data/streamlit_data/{list_nm[i]}/{list_nm[i]}.csv'
    total.to_csv(file_name, index = False)
    print(f'{list_nm[i]} 파일 생성 완료')        
    
# 실행 후 시간 기록
end_time = time.time()  

# 실행 시간 계산
execution_time = end_time - start_time
print(f"실행 시간: {execution_time} 초")   

1
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          455
동리명         50632
geometry       54
dtype: int64

60133
2
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          375
동리명         42413
geometry       24
dtype: int64

110596
3
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0

785904
22
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          183
동리명         12992
geometry        7
dtype: int64

802291
23
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          232
동리명         12512
geometry        5
dtype: int64

817598
24
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동   

아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

437870
7
아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

532523
8
아파트 전월세
------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

2153680
25
아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

2253314
26
아파트 전월세
----------------------------------------------------------------------------------------

오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

39097
8
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

45063
9
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0

오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

118312
28
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지          0
동리명         0
시군구         0
월           0
일           0
전용면적        0
중개사소재지      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

120496
29
오피스텔 매매
----------------------------------------------------------------------------------------------------
거래금액        0
거래유형        0
건축년도        0
년           0
단지       

오피스텔 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

144904
11
오피스텔 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

160437
12
오피스텔 전월세
---------------------------------------------------

----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

461448
29
오피스텔 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
단지          0
동리명         0
보증금         0
시군구         0
월           0
월세          0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

477288
30
오피스텔 전월세
------------------------------------------------------------

### 아파트 오피스텔 최신데이터를 전처리해줍니다 (202311)

In [48]:
# # 실행 전 시간 기록
# start_time = time.time()

# # 국토교통부 데이터 세트
# list_nm =['apt_trade','apt_rent','offi_trade','offi_rent']
# names = ['아파트 매매','아파트 전월세','오피스텔 매매','오피스텔 전월세']

# directory = '0_data/streamlit_data/cleaning'
# # 폴더가 없으면 폴더 생성
# if not os.path.exists(directory):
#     os.makedirs(directory)
#     print(f"폴더 '{directory}'가 생성되었습니다.")

# # 4개 데이터 세트 반복문
# for i in range(len(list_nm)):
    
#     total = pd.DataFrame()
    
#     # 각 타입별 모든 데이터 조회
#     dir_list = f'./0_data/streamlit_data/{list_nm[i]}_*.csv'
#     file_list = sorted(glob.glob(dir_list))

#     if os.name == 'nt':  # 윈도우 플랫폼
#         file_list = [file.replace('\\', '/') for file in file_list]
    
#     for k in range(len(file_list)):
#         print(k + 1)     
        
#         try:
#             df = pd.read_csv(file_list[k], thousands=r',')
        
#             clean_df = molit_data_cleaning(df,names[i])
#             clean_df = clean_df.drop(columns=['geometry'])
        
#             total = pd.concat([total,clean_df])
#             print(len(total))
#         except Exception as e:
#             print(f"예외가 발생했습니다: {e}")
#             break
            
#     file_name = f'0_data/streamlit_data/cleaning/{list_nm[i]}_202311.csv'
#     total.to_csv(file_name, index = False)
#     print(f'{list_nm[i]}_202311.csv 파일 생성 완료')        
    
# # 실행 후 시간 기록
# end_time = time.time()  

# # 실행 시간 계산
# execution_time = end_time - start_time
# print(f"실행 시간: {execution_time} 초")   

1
아파트 매매
----------------------------------------------------------------------------------------------------
거래금액            0
거래유형            0
건축년도            0
전용면적            0
법정동             0
법정동코드           0
아파트             0
층               0
년               0
월               0
일               0
시도명             0
시군구명          182
동리명         18588
geometry       22
dtype: int64

22636
apt_trade_202311.csv 파일 생성 완료
1
아파트 전월세
----------------------------------------------------------------------------------------------------
갱신요구권사용     0
건축년도        0
계약구분        0
계약기간        0
년           0
동리명         0
보증금액        0
아파트         0
월           0
월세금액        0
일           0
전용면적        0
종전계약보증금     0
종전계약월세      0
지번          0
법정동시군구코드    0
층           0
시도명         0
시군구명        0
주소          0
법정동코드       0
법정동읍면동코드    0
geometry    0
dtype: int64

69764
2
예외가 발생했습니다: No columns to parse from file
apt_rent_202311.csv 파일 생성 완료
1
오피스텔 매매
----------------------------------

### 부동산 각 시군구별 년월 평균 거래 금액 데이터를 만들어줍니다

In [49]:
vis_trade_rent = pd.read_csv('0_data/vis/vis_trade_rent.csv')
vis_trade_rent

,시도명,시군구명,년,월,구분,count,mean,거래날짜,타입
0,경기도,가평군,2021,1,매매,31,20251,2021-01-01,apt
1,경기도,가평군,2021,2,매매,37,20417,2021-02-01,apt
2,경기도,가평군,2021,3,매매,44,18307,2021-03-01,apt
3,경기도,가평군,2021,4,매매,26,19673,2021-04-01,apt
4,경기도,가평군,2021,5,매매,27,19418,2021-05-01,apt
...,...,...,...,...,...,...,...,...,...
39683,충청북도,충주시,2023,6,전세,1,8000,2023-06-01,offi
39684,충청북도,충주시,2023,7,월세,2,750,2023-07-01,offi
39685,충청북도,충주시,2023,8,월세,4,500,2023-08-01,offi
39686,충청북도,충주시,2023,10,월세,4,400,2023-10-01,offi


In [50]:
vis_trade_rent = vis_trade_rent.rename(columns={'mean':'거래금액'})
vis_trade_rent.reset_index(drop=True,inplace= True)
vis_trade_rent

,시도명,시군구명,년,월,구분,count,거래금액,거래날짜,타입
0,경기도,가평군,2021,1,매매,31,20251,2021-01-01,apt
1,경기도,가평군,2021,2,매매,37,20417,2021-02-01,apt
2,경기도,가평군,2021,3,매매,44,18307,2021-03-01,apt
3,경기도,가평군,2021,4,매매,26,19673,2021-04-01,apt
4,경기도,가평군,2021,5,매매,27,19418,2021-05-01,apt
...,...,...,...,...,...,...,...,...,...
39683,충청북도,충주시,2023,6,전세,1,8000,2023-06-01,offi
39684,충청북도,충주시,2023,7,월세,2,750,2023-07-01,offi
39685,충청북도,충주시,2023,8,월세,4,500,2023-08-01,offi
39686,충청북도,충주시,2023,10,월세,4,400,2023-10-01,offi


In [51]:
vis_trade_rent2 = vis_trade_rent[['시군구명','년','월','구분','거래금액','타입']]
vis_trade_rent2

,시군구명,년,월,구분,거래금액,타입
0,가평군,2021,1,매매,20251,apt
1,가평군,2021,2,매매,20417,apt
2,가평군,2021,3,매매,18307,apt
3,가평군,2021,4,매매,19673,apt
4,가평군,2021,5,매매,19418,apt
...,...,...,...,...,...,...
39683,충주시,2023,6,전세,8000,offi
39684,충주시,2023,7,월세,750,offi
39685,충주시,2023,8,월세,500,offi
39686,충주시,2023,10,월세,400,offi


In [52]:
legal_geo_info_3

,법정동시군구코드,시도명,시군구명
0,11110,서울특별시,종로구
87,11140,서울특별시,중구
161,11170,서울특별시,용산구
197,11200,서울특별시,성동구
214,11215,서울특별시,광진구
...,...,...,...
19634,48870,경상남도,함양군
19748,48880,경상남도,거창군
19854,48890,경상남도,합천군
20063,50110,제주특별자치도,제주시


In [53]:
property_trade_map = pd.merge(vis_trade_rent2,legal_geo_info_3,
                            on = '시군구명',
                            how = 'left')
property_trade_map

,시군구명,년,월,구분,거래금액,타입,법정동시군구코드,시도명
0,가평군,2021,1,매매,20251,apt,41820,경기도
1,가평군,2021,2,매매,20417,apt,41820,경기도
2,가평군,2021,3,매매,18307,apt,41820,경기도
3,가평군,2021,4,매매,19673,apt,41820,경기도
4,가평군,2021,5,매매,19418,apt,41820,경기도
...,...,...,...,...,...,...,...,...
61340,충주시,2023,6,전세,8000,offi,43130,충청북도
61341,충주시,2023,7,월세,750,offi,43130,충청북도
61342,충주시,2023,8,월세,500,offi,43130,충청북도
61343,충주시,2023,10,월세,400,offi,43130,충청북도


In [54]:
property_trade_map.rename(columns={'법정동시군구코드':'SIG_CD'},inplace=True)
property_trade_map

,시군구명,년,월,구분,거래금액,타입,SIG_CD,시도명
0,가평군,2021,1,매매,20251,apt,41820,경기도
1,가평군,2021,2,매매,20417,apt,41820,경기도
2,가평군,2021,3,매매,18307,apt,41820,경기도
3,가평군,2021,4,매매,19673,apt,41820,경기도
4,가평군,2021,5,매매,19418,apt,41820,경기도
...,...,...,...,...,...,...,...,...
61340,충주시,2023,6,전세,8000,offi,43130,충청북도
61341,충주시,2023,7,월세,750,offi,43130,충청북도
61342,충주시,2023,8,월세,500,offi,43130,충청북도
61343,충주시,2023,10,월세,400,offi,43130,충청북도


In [55]:
property_trade_map[(property_trade_map['년'] == 2021) & (property_trade_map['월'] == 9)]

,시군구명,년,월,구분,거래금액,타입,SIG_CD,시도명
8,가평군,2021,9,매매,20353,apt,41820,경기도
44,고양시 덕양구,2021,9,매매,57139,apt,41281,경기도
80,고양시 일산동구,2021,9,매매,61170,apt,41285,경기도
116,고양시 일산서구,2021,9,매매,52787,apt,41287,경기도
152,과천시,2021,9,매매,195850,apt,41290,경기도
...,...,...,...,...,...,...,...,...
61181,청주시 청원구,2021,9,월세,683,offi,43114,충청북도
61182,청주시 청원구,2021,9,전세,9416,offi,43114,충청북도
61253,청주시 흥덕구,2021,9,월세,639,offi,43113,충청북도
61254,청주시 흥덕구,2021,9,전세,13786,offi,43113,충청북도


In [56]:
property_trade_map2 = property_trade_map[property_trade_map['시도명'] == '부산광역시']
property_trade_map3 = property_trade_map2[property_trade_map2['년'] == 2022]
property_trade_map3

,시군구명,년,월,구분,거래금액,타입,SIG_CD,시도명
3196,남구,2022,1,매매,29689,apt,26290,부산광역시
3200,남구,2022,2,매매,29291,apt,26290,부산광역시
3204,남구,2022,3,매매,32384,apt,26290,부산광역시
3208,남구,2022,4,매매,31796,apt,26290,부산광역시
3212,남구,2022,5,매매,33457,apt,26290,부산광역시
...,...,...,...,...,...,...,...,...
59223,중구,2022,10,전세,10746,offi,26110,부산광역시
59229,중구,2022,11,월세,1225,offi,26110,부산광역시
59235,중구,2022,11,전세,10317,offi,26110,부산광역시
59241,중구,2022,12,월세,993,offi,26110,부산광역시


In [57]:
directory = '0_data/streamlit_data/geoservice'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"디렉토리 '{directory}'가 생성되었습니다.")
else:
    print(f"디렉토리 '{directory}'는 이미 존재합니다.")

디렉토리 '0_data/streamlit_data/geoservice'는 이미 존재합니다.


In [58]:
property_trade_map.to_csv('0_data/streamlit_data/geoservice/property_trade_map.csv',index=False)

### 각 시도별 위경도 구하기

In [59]:
# cities = legal_geo_info_3['시도명'].unique().tolist()
# cities

In [60]:
# api_key = '4512a1478c5eaeeef7f1f3c1b88218e7'

# # 주소 -> 위경도 추출하는 함수
# def addr_to_lat_lon(address):
#     url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
#     headers = {"Authorization": "KakaoAK " + api_key}
#     result = json.loads(str(requests.get(url, headers=headers).text))
#     match_first = result['documents'][0]['address']
#     return float(match_first['x']), float(match_first['y'])

# # 위경도 -> 주소 추출하는 함수
# def lat_lon_to_addr(lon,lat):
#     url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lon}&y={lat}'
#     headers = {"Authorization": "KakaoAK " + api_key}
#     result = json.loads(str(requests.get(url, headers=headers).text))
#     match_first = result['documents'][0]['address_name']
#     return str(match_first)

In [61]:
# for city in cities:
#     print(addr_to_lat_lon(city))

In [62]:
# # 빈 리스트 생성
# data = []

# # cities 리스트에 있는 각 city 값에 대해 addr_to_lat_lon() 함수 호출 및 결과 기록
# for city in cities:
#     result = addr_to_lat_lon(city)  # addr_to_lat_lon() 함수는 주소를 경도와 위도로 변환하는 함수입니다.
#     print(result)
#     data.append({'sig_nm': city, 'lat': result[0], 'long': result[1]})

# # 데이터프레임 생성
# df = pd.DataFrame(data)

# df

In [63]:
# df.to_csv('0_data/streamlit_data/geoservice/sig_lat_lon.csv',index=False)

In [64]:
# legal_info_b.to_csv('0_data/streamlit_data/legal_info_b.csv')